Task1: Managing Conversation History with Summarization

In [171]:
from openai import OpenAI

In [172]:
client = OpenAI(api_key="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",    base_url="https://api.groq.com/openai/v1"
)

In [173]:
#creating empty storage
history= []
count =0
model ="llama-3.1-8b-instant"

In [174]:
#adding text into the storage
def add_mssg(role, content):
    global history, count
    history.append({"role": role, "content": content})
    count+=1

In [175]:
#function keeps onlylast n messages in history .
def truncate_by_turns(n=4):
  global history
  history = history[-n:]


In [176]:
#it Keeps only the latest messages within the set character limit.
def truncate_by_chars(limit=300):
    global history
    text=""
    newhist=[]
    for msg in reversed(history):
        if len(text) +len(msg["content"]) <=limit:
            newhist.insert(0, msg)
            text+= msg["content"]
        else:
            break
    history =newhist
    output(history, f"After truncation by chars (limit={limit})")

In [177]:
def summarize_every_k_runs(k=3):
    global history, count, model
    if count% k ==0:
        summary_prompt = [{"role": "system", "content": "Summarize the conversation briefly."},{"role": "user", "content": str(history)} ]
        response =client.chat.completions.create(model=model, messages=summary_prompt)
        summary= response.choices[0].message.content
        history =[{"role": "system", "content": f"Summary: {summary}"}]
        output(history, f"After summarization (k={k})")

In [178]:
#to display the output neatly and understandable

def output(history, title="Conversation History"):
    print(f"\n{title}")
    for msg in history:
        print(f"Role: {msg['role']}")
        print(f"Content: {msg['content']}\n")

In [179]:
add_mssg("me", "Hi")
add_mssg("friend", "Hello!")
add_mssg("me","tell me a joke.")
print("Before summarization:\n",history)
summarize_every_k_runs(k=3)
print("\nAfter summarization:\n",history)




Before summarization:
 [{'role': 'me', 'content': 'Hi'}, {'role': 'friend', 'content': 'Hello!'}, {'role': 'me', 'content': 'tell me a joke.'}]

After summarization (k=3)
Role: system
Content: Summary: You started a conversation by saying "Hi" and your friend replied with "Hello!" You then asked your friend to "tell me a joke," but the conversation ended there.


After summarization:
 [{'role': 'system', 'content': 'Summary: You started a conversation by saying "Hi" and your friend replied with "Hello!" You then asked your friend to "tell me a joke," but the conversation ended there.'}]


In [180]:
add_mssg("me","Hi, can you help me with my homework?")
add_mssg("friend", "Sure! which subject?")
add_mssg("me", "Maths.")
print("History after 3 messages:\n", history)
print("messagecount:", count)

History after 3 messages:
 [{'role': 'system', 'content': 'Summary: You started a conversation by saying "Hi" and your friend replied with "Hello!" You then asked your friend to "tell me a joke," but the conversation ended there.'}, {'role': 'me', 'content': 'Hi, can you help me with my homework?'}, {'role': 'friend', 'content': 'Sure! which subject?'}, {'role': 'me', 'content': 'Maths.'}]
messagecount: 6


In [181]:
truncate_by_turns(n=2)
print("\nAfter truncation by turns:\n", history)



After truncation by turns:
 [{'role': 'friend', 'content': 'Sure! which subject?'}, {'role': 'me', 'content': 'Maths.'}]


In [182]:
summarize_every_k_runs(k=2)
print("\nsummary:\n",history)


After summarization (k=2)
Role: system
Content: Summary: You wanted information about Maths. However, there's no further conversation to summarise yet.


summary:
 [{'role': 'system', 'content': "Summary: You wanted information about Maths. However, there's no further conversation to summarise yet."}]


In [183]:
add_mssg("me", "Okay, let's solve an equation together.")
add_mssg("friend", "Solve 2x + 5 = 15")
truncate_by_chars(limit=60)
print("\nAfter truncation by characters:\n", history)



After truncation by chars (limit=60)
Role: me
Content: Okay, let's solve an equation together.

Role: friend
Content: Solve 2x + 5 = 15


After truncation by characters:
 [{'role': 'me', 'content': "Okay, let's solve an equation together."}, {'role': 'friend', 'content': 'Solve 2x + 5 = 15'}]


In [184]:
summarize_every_k_runs(k=2)
print("\nsummary:\n",history)


After summarization (k=2)
Role: system
Content: Summary: I'm ready to help, but it seems like you've already started the conversation with your friend. 

To summarize, your friend provided the equation 2x + 5 = 15, and we're planning to solve it together.


summary:
 [{'role': 'system', 'content': "Summary: I'm ready to help, but it seems like you've already started the conversation with your friend. \n\nTo summarize, your friend provided the equation 2x + 5 = 15, and we're planning to solve it together."}]


Task2: JSON Schema Classification & Information Extraction

In [111]:
import json
import re

In [112]:
schema ={"name": "string","email":"string","phone":"string","location": "string","age": "string"}

In [113]:
#Ask AI to fetch JSON info from chat, clean it, and return as dict; else show raw if fail.
def info_extract(chat):
    response = client.chat.completions.create(model="llama-3.1-8b-instant",
        messages=[{"role": "system", "content": "Extract the following info in JSON format: name, email, phone, location, age. Respond ONLY in valid JSON, no extra text, no markdown."},
{"role": "user", "content": chat} ],
        max_tokens=200 )
    content = response.choices[0].message.content
    # Remove extra text
    match= re.search(r"\{.*\}", content, re.DOTALL)
    if match:
        content =match.group(0)
      #if parsing fails
    try:
        return json.loads(content)
    except:
        return {"error":"couldn't parse JSON", "raw": content}


In [114]:
chats = [
  "Hey! I’m Tanu. Just moved to Delhi last month. You can email me at tanu123@gmail.com and call me on 9876543210. By the way, I’m 22 years old.",
    "Hi there, this is Raj from Mumbai. If you need to reach me, my email is raj.kumar@example.com and my phone number is 9123456789.",
  "Hello! I’m Priya living in Bangalore. My email is priya_b@example.com and my number is 9988776655. I’m 30 years old.",
    "Hey! My name’s Ankit. I live in Pune and you can contact me via email ankit.pune@example.com or phone 9122334455. I’m 28 years old.",
    "Hi! I’m Sneha from Hyderabad. You can reach me at sneha_hyderabad@gmail.com and phone 9871122334. I’m 26. Nice to meet you!"
    ]


In [115]:
#Loops through chats andextracts info
for i, chat in enumerate(chats, 1):
    info =info_extract(chat)
    print(f"info frm chat {i}:")
    print(json.dumps(info, indent=2))
    print("*" * 50)

info frm chat 1:
{
  "name": "Tanu",
  "email": "tanu123@gmail.com",
  "phone": "9876543210",
  "location": "Delhi",
  "age": 22
}
**************************************************
info frm chat 2:
{
  "name": "Raj",
  "email": "raj.kumar@example.com",
  "phone": "9123456789",
  "location": "Mumbai",
  "age": null
}
**************************************************
info frm chat 3:
{
  "name": "Priya",
  "email": "priya_b@example.com",
  "phone": "9988776655",
  "location": "Bangalore",
  "age": 30
}
**************************************************
info frm chat 4:
{
  "name": "Ankit",
  "email": "ankit.pune@example.com",
  "phone": "9122334455",
  "location": "Pune",
  "age": "28"
}
**************************************************
info frm chat 5:
{
  "name": "Sneha",
  "email": "sneha_hyderabad@gmail.com",
  "phone": "9871122334",
  "location": "Hyderabad",
  "age": "26"
}
**************************************************
